## Audio experiments

In [185]:
#>flags -d:blas=cblas -p:~/code/misc/jnim

# Original Python code:
# srate = 11000
# dur = 1000
# rr = np.arange(dur*2)
# wave = np.sin(2*np.pi*400*rr/srate) * np.sin(0.5*np.pi*rr/dur)
# plt.plot(rr, wave)
# Audio(wave, rate=srate, autoplay=True)
    
import sequtils, std/[strutils, math, base64, strformat, times, os]
import arraymancer, pixie, jupyternimpkg/display
import wavfile, vorbis, fft

proc pixieHTML(img: Image): string =
    return fmt"""<img src="data:image/png;base64, {img.encodeImage(PngFormat).encode}" />"""

proc plot1D(img: Image, arr: openArray[float]): Image =
    var path: seq[string] = @[]
    
    let amax = arr[arr.maxIndex]
    let amin = arr[arr.minIndex]
    
    let mp = float(img.height)/(amax - amin)
    
    # For long arrays, draw <wmul> overlapping
    let wmul = 8
    if arr.len < img.width * wmul or false:
        for i in 0..<arr.len:
            let arr_v = arr[i]
            path.add(fmt"{int(float(i)*float(img.width)/float(arr.len))} {int((arr_v-amin) * mp)} ")
    else:
        for i in 0..<img.width * wmul:
            let arr_v = arr[int(float(i)*float(arr.len)/float(img.width)) div wmul]
            path.add(fmt"{i div wmul} {int((arr_v-amin) * mp)} ")
        
    img.strokePath(fmt"M {path[0]} L " & path.join(""), rgba(0, 0, 128, 255), strokeWidth = 1)
    return img

proc basicSine(srate: int): Tensor[system.float] =
    let srate = float(srate)
    let dur = 1000
    let rr_dur = dur * 2
    let rr = toSeq(0..<rr_dur).mapIt(it.float).toTensor()
    return sin(rr*(2.0*PI*440.0/srate)) *. sin(rr*(0.5*PI/float(dur)))


template benchmark(benchmarkName: string, code: untyped) =
  block:
    let t0 = epochTime()
    code
    let elapsed = epochTime() - t0
    let elapsedStr = elapsed.formatFloat(format = ffDecimal, precision = 3)
    echo "CPU Time [", benchmarkName, "] ", elapsedStr, "s"

let wsample = loadWav("sample.wav")
#html wsample.toHTML(false)
#html pixieHTML(newImage(500,100).plot1D(wsample.toFloat))

let srate = 11025
let wave = basicSine(srate).toSeq
#html Audio(wave, srate, false)
#html pixieHTML(newImage(400, 100).plot1D(wave))

#let vsample = loadVorbis("sample.ogg")
#html vsample.toHTML(false)
#var vsf = wsample.toFloat.mapIt(complex(it))
var vsf = wave.mapIt(complex(it))
benchmark "fft":
    fft(vsf)
#ifft(vsf)
let fftr = vsf.mapIt(-abs(it.re))
benchmark "showimage":
    html pixieHTML(newImage(500,100).plot1D(fftr[0..fftr.len div 2]))

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAfQAAABkCAYAAABwx8J9AAAVvUlEQVR4Ae2dC7AkVXnHf+deHm5SKwkI+AxKLSBieMsjunRfWIKKUjFlqDKapZIqJaVJAZaSWAaI62PDIywiYjaLBFyCriBZAvKSx55ZBOSxQogrLCS8wkMsJRsWRPDeSX1nZu5l787jTp/uM326v1/VFOz0nHO+8//3d76e7p6+4yiKoiiKkiOnngLp4WDX5NjpQLSgK4qiKEquJAmwLdibc+12AFrQFUVRFCVXpKCbeWB/kGu3A9CCriiKoii54gr6fLDX5drtALSgK4qiKEquSEFnO7DX5NrtALSgK4qiKEquSEE3vwv2qly7HYAWdEVRFEXJFSnovA7slbl2OwAt6IqiKIqSK66g7wz2ily7HYAWdEVRFEXJFSno5g1gL8u12wFoQVcURVGUXHEF/S1gv5NrtwPQgq4oiqIouSIFnV3AXpprtwPQgq4oiqIouSIF3bwN7Mpcux2AFnRFURRFyRUp6CwAe3Gu3Q5AC7qiKIqi5Ior6HuAvTDXbgegBV1RFEVRckUKutkT7Ipcux2AFnRFURRFyRUp6LwT7PJcux2AFnSlTqAoilI8SQLsA/b84seaYXzmfxVFURRF8UcKutkP7Nf8+5o7WtAL49RTID0c7JrChlAURVFKiBR0DgT71ZDBaUEvDDHUvAbsTYUNoSiKopQQWf85GOzZIYPTgl4YYqiZD/b6woZQFEVRSohb/w8Fe2bI4LSgF4YzdDuw1xQ2hKIoilJCZP3nPWBPDxmcFvTCEEPN9mCvKmwIRVEUpYS49f8wsF8JGZwW9MIQQ9kJ7OrChlAURVFKiKz/Rm6K/mLI4LSgF4Yz9A1gLy9sCEVRFKWEyPo/tgjWfCFkcFrQC0MMNW8Gu6qwIRRFUZQS4tb/CUiWgp0MFaAW9MIQQ3kr2EsLG0JRFEUpIbL+S0HfdDrc/kqoALWgF4YYyq5gVxY2hKIoilJCZP2Xgr7NmWBfChWgFvTCEEPZDexFhQ2hKIqilBBZ/6Wgv3w23PpiqAC1oBeGGMo7wF5Q2BCKoihKCZH1Xwr61Fdh7fOhAtSCXhhiKO8Eu7ywIRRFUZQSIuu/FPSxr4H931ABakEvDDE0/J/PUxRFUUaNrP9S0CfPh8YvQ0VT4oIuf60smQBrQ4mRL2Io+4f+83nVIXb/FT/U/xwgXk49BZIJsDbOOSQJmAmYWg6Nn4eaw3iogYbHCbIN2JuHb1sGJH4OBrusDNHEh+gXs/+KH+p/vYndfxf/BJgVYJ8J5WXJCzrzwN4YSox8EUM5FOxZ+fZbF5x+Efuv+KH+15vY/Zf4paBPXQiNp0J5WfaC/ltgbwglRr6IobwH7On59lsXnH4R+6/4Ebv/7pTxEWDX+OlQV2L3X+KXgt68GBpPhHKx5AXdzAd7bSgx8sUZmoL9cr791gWnX8T+K37E7r/E7w5IIz3DOGpEv5jz38Uvp9xXgn00lJplL+jbgf1+KDHyRQxlEdgl+fZbF0S/mP1X/Ijdf4mf14K9zk+HuiL6xZz/Ln65Ke5SaPx3KBdLXtDZHuxVocTIFzF07HBIvgx2Kt++64DoF7P/ih+x+z8d/9V+OtSVaf0iXv+loJvvgH0olIslL+jmdWBXhxIjX2SHFEM3nQG3v5Jv33XA6Rex/4ofsfvv4t8Z7L/56VBXnH4R57+LfwK4HOwDoVwseUFHEuKKUGLkS8fQ5llw60v59l0HRL+Y/Vf8iN1/id+8CexlfjrUFdEv5vyX+KWgN68Auz6Ui2Uv6K8He3koMfKlY+jkMrj1hXz7rgOiX8z+K37E7r/Eb3YB+20/HeqK6Bdz/kv8rqBfCfb+UC6WvKC7I9xVocTIl2lDz4O1G/Ptuw44/eQbTqT+K37E7r/EzwKwK/10qCuiX8z57+KXU+5Xgb03lItlL+hvifcIt2No83xo/DKUodXB6Rex/4ofsfsv8bMH2H/x06GuiH4x57+LfwKa14BdF8rFkhd03gr2X0OJkS8dQ8M+y7c6iH4x+6/4Ebv/Lv69wK7w06GuOP0qsP5zHdi7QrlY8oJudgX7rVBi5IvskHKENnUBNII9y7c6OP0i9l/xI3b/Xfz7gj3fT4e64vSLOP9d/HLK/Uawd4RysewFXa5BXRRKjHzpGOoe/fc/+fZdB5x+Efuv+BG7/y7+A8Ge66dDXXH6RZz/Ln455X4z2NtCuVjygu6uQV0YSox8mTZ0JTQey7fvOiD6xey/4kfs/kv85hCwZ/vpUFdEv5jzX+KXgs4asGtDuVj2gv52sBeEEiNfOoY2gz76rzqIfjH7r/gRu/8u/oVgz/DToa44/aqw/luwjVAulrygG7mpZHlxYlAgSQJmAsx3wW4ocKCK4vTbC+zyik5Q6Uvs/kv8pGC/0neaSg9Ev5jz38U/Ac1bwDZ6TDJ3xnPvMTdEEPYG+43cugyKxC+GTl0BjZ8GHboSiH4x+6/4Ebv/Lv5FYJf46VBXnH4VWP+1oLdxhu4D9uvtNyJD4peC3lwNjf+MLPgSIPrF7L/iR+z+S/xjh0OyFOyknxZ1RPSLOf8lfrf+6zf0Fk6Q/cGe2/p3bLj45ZT71WDvjS360eP0i9h/xY/Y/XfxT8CLZ8JtL/tpUUecfhHnv4tfT7nPIILwLrDnzLwXExK/GMq1YO+JKfJyIPrF7L/iR+z+S/yS/1ufA/YFPy3qiOgXc/5L/OK/fkNvI4JwENiz229EhsQvhnID2DsjC74EiH4x+6/4Ebv/Er/L//PAbvTToo6IfjHnv8Qv/mtBb+MEORTsme03IsPFPwHINZTbIgu+BDj9IvZf8SN2/138kv//BPYXflrUEadfxPnv4tdT7jOIILwH7Okz78WExC+GYsGujSnyciD6xey/4kfs/kv8kv+T34TGz/y0qCOiX8z5L/GL//oNvY0IwkKwS9tvRIbE7wyVgh7sd4jVQfSL2X/Fj9j9l/hd/l8EjSf9tKgjol/M+S/xO//lDG2w9b/kv0M38mCGL8W5O4/G0Org9IvYf8WP2P138csp10ug8ZifFnXE6Rdx/rv4xX8t6C1EEI6I98EMEn94Q6uD6Bez/4ofsfsv8Uv+m1VgH/bToo6IfjHnv8Qv/mtBbyOCuAczfBFss/1mREj84Q2tDqJfzP4rfsTuv8Qv+c/3wD7gp0UdEf1izn+JX/zXgt6mI8gvl8KdET5pqRN/WEOrQ0e/WP1X/Ijd/078XAlWnxQ5NB39Ys3/Tvxh1/+yX0OfgMl/gNtfGXp/GDmjMbQ6dPSL1X/Fj9j978TfvEafFJmFjn6x5n8nfi3obTqCbHMm2Jfab0ZEJ/6whlaHjn6x+q/4Ebv/nfib14O920+LOtLRL9b878Qfdv2P4Bv6y2fDrS/Gt0uPxtDq0NEvVv8VP2L3vxN/8yawd/hpUUc6+sWa/534taC3SReBSWBqGazd1H4zIkZjaHWI3f8yQkSki8AkMLUM1m6KKPA2SQJmAmiAXdt+U5kzVfG/eQvYxpyn7cm4Z/sCSd8H5t0wdi7YjQUOVBCdhA5raHWI3X/Fj9j9n85/G3JBrw6V8V8LeovkGDAHwW++Dmufa70XE9MJHdTQ6hC7/4ofsfuv+e+H+p+FEn9DT/4YzP7QPB8av8gyudGiCe1H7P4rfsTuv+a/H+p/Fspc0I8FszeY5WCfzTK50aIJ7Ufs/it+xO6/5r8f6n8WSlzQ04+CeQdMXgCNZ7JMbrRoQvsRu/+KH7H7r/nvh/qfhTIX9MVgdofmN6HxVJbJjRZNaD9i91/xI3b/Nf/9UP+zUOaC/hfA28BcDPaJLJMbLZrQfsTuv+JH7P5r/vuh/mehxAU9OR74HTCrwD6WZXKjRRPaj9j9V/yI3X/Nfz/U/yyUuKCnfwVma5j6d2g8kmVyo0UT2o/Y/Vf8iN1/zX8/1P8slLigJyeC+TWYG8E+nGVyo0U

CPU Time [fft] 0.001s
CPU Time [showimage] 0.018s
